In [1]:
import AdventOfCode (dayLines, parsed)
import qualified Data.Attoparsec.ByteString.Char8 as A
import Data.Bits

input <- dayLines 14

In [2]:
import Numeric
import Data.Char
showBinary n = showIntAtBase 2 intToDigit n ""
readBinary s = fst . head $ readInt 2 isDigit digitToInt s
pad s = replicate (36 - length s) '0' <> s

In [3]:
masked :: Int -> String -> Int
masked int mask = let
    bin = pad $ showBinary int
    apply = zipWith (\x y -> if y == 'X' then x else y) bin mask
    in readBinary apply

In [4]:
{-# LANGUAGE OverloadedStrings #-}

import Data.ByteString.UTF8 (toString)

data Instr = Mem Int Int | Mask String deriving (Eq, Show)


parseMask = do
    "mask = " *> (Mask . toString <$> A.take 36)

parseMem = do
    A.string "mem"
    loc <- "[" *> A.decimal <* "]"
    A.string " = "
    Mem loc <$> A.decimal

parseInstr = A.choice [parseMask, parseMem]

instructions = map (parsed parseInstr) input

In [5]:
import qualified Data.Map.Strict as M

data MachineState = MachineState { currentMask :: String, currentMemory :: M.Map Int Int } deriving (Eq, Show)


executeInstruction1 :: MachineState -> Instr -> MachineState
executeInstruction1 ms (Mask mask) = ms { currentMask = mask }
executeInstruction1 (MachineState mask memory) (Mem loc val) = let
    memory' = M.insert loc (masked val mask) memory
    in MachineState mask memory'

In [6]:
import Data.List (foldl')

final1 = foldl' executeInstruction1 (MachineState "" M.empty) instructions

In [7]:
sum $ M.elems $ currentMemory final1

10452688630537

In [8]:
anchorFloating acc [] = [acc]
anchorFloating acc ('X':xs) = anchorFloating (acc <> "1") xs <> anchorFloating (acc <> "0") xs
anchorFloating acc (x:xs) = anchorFloating (acc <> [x]) xs

In [9]:
interpretMemoryMask int mask = let
    bin = pad $ showBinary int
    in zipWith (\x y -> case y of
        'X' -> 'X'
        '0' -> x
        '1' -> '1') bin mask

In [10]:
executeInstruction2 :: MachineState -> Instr -> MachineState
executeInstruction2 ms (Mask mask) = ms { currentMask = mask }
executeInstruction2 (MachineState mask memory) (Mem loc val) = let
    floatingMemoryLoc = interpretMemoryMask loc mask
    locations = map readBinary $ anchorFloating "" floatingMemoryLoc
    memory' = foldl' (\mem location -> M.insert location val mem) memory locations
    in MachineState mask memory'

In [11]:
final2 = foldl' executeInstruction2 (MachineState "" M.empty) instructions
sum $ M.elems $ currentMemory final2

2881082759597